In [1]:
import os
from typing import Iterable, Dict

import csv

import numpy as np
from matplotlib.image import imread
from tqdm import tqdm
from PIL import Image

import seaborn as sns
import pandas as pd
import scipy.io

import cv2


In [2]:
!pwd

/src/drone_causality/fly_to_target_dataset


In [7]:
# CONVERTING MAT TO CSV FILE 
def mat_to_csv(dataset_directory):

    # Iterate through directories in the dataset directory
    for i in range(len(os.listdir(dataset_directory))):
        directory = os.path.join(dataset_directory, str(i + 1))
        csv_file_path = os.path.join(directory, 'data_out.csv')
        
        # Check if the directory exists and data_out.csv does not exist
        if os.path.exists(directory) and (not os.path.exists(csv_file_path)):
            mat_file_path = os.path.join(directory, 'Vel.mat')

            # Check if the Vel.mat file exists in the directory
            if os.path.exists(mat_file_path):
                # Load the .mat file
                data = scipy.io.loadmat(mat_file_path)
                df = pd.DataFrame(data['V'], columns=['vx', 'vy', 'vz', 'omega_z'])

                # Save to CSV in the same directory
                df.to_csv(csv_file_path, index=False)

            else:
                print(f"'Vel.mat' file not found in {directory}")
        else:
            print(f"Directory {directory} does not exist or 'data_out.csv' already exists in directory {directory}")

In [8]:
# Directory containing the datasets
dataset_directory = 'dataset' 

original_dataset_directory = 'original_dataset'

mat_to_csv(dataset_directory)
mat_to_csv(original_dataset_directory)

Directory dataset/1 does not exist or 'data_out.csv' already exists in directory dataset/1
Directory dataset/2 does not exist or 'data_out.csv' already exists in directory dataset/2
Directory dataset/3 does not exist or 'data_out.csv' already exists in directory dataset/3
Directory dataset/4 does not exist or 'data_out.csv' already exists in directory dataset/4
Directory dataset/5 does not exist or 'data_out.csv' already exists in directory dataset/5
Directory dataset/6 does not exist or 'data_out.csv' already exists in directory dataset/6
Directory dataset/7 does not exist or 'data_out.csv' already exists in directory dataset/7
Directory dataset/8 does not exist or 'data_out.csv' already exists in directory dataset/8
Directory dataset/9 does not exist or 'data_out.csv' already exists in directory dataset/9
Directory dataset/10 does not exist or 'data_out.csv' already exists in directory dataset/10
Directory dataset/11 does not exist or 'data_out.csv' already exists in directory datase

In [9]:
# 30 % of each trajecrory as hovering 

root = "dataset"
file_ending = 'png'
file_name = "data_out.csv"

for i in range(len(os.listdir(root))):
    directory = i + 1
    print("Processing directory", directory)

    csv_file_name = f"{root}/{str(directory)}/{file_name}"
    labels = np.genfromtxt(csv_file_name, delimiter=',', skip_header=1, dtype=np.float32)
    diff = int((30 * len(labels))/100)
    print("diff_labels", diff)
    print("labels len", labels.shape)
    
    velocity_values = [0.0, 0.0, 0.0, 0.0]
    
    with open(csv_file_name, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        for i in range(diff):
            csvwriter.writerow(velocity_values)
   

    n_images = len([fn for fn in os.listdir(f"./{root}/{str(directory)}") if file_ending in fn])
    print("Number of images:", n_images)
    
    img_file_name = f"{root}/{str(directory)}/Image{str(n_images)}.png"
    img = Image.open(img_file_name)
    for i in range(1, diff + 1):
       
        new_filename = os.path.join(root,str(directory))
        fname =  f"Image{str(i + n_images)}.png"
        file_path = os.path.join(new_filename, fname)
        
        img.save(file_path)
        

Processing directory 1
diff_labels 177
labels len (593, 4)
Number of images: 593
Processing directory 2
diff_labels 162
labels len (542, 4)
Number of images: 542
Processing directory 3
diff_labels 149
labels len (497, 4)
Number of images: 497
Processing directory 4
diff_labels 156
labels len (520, 4)
Number of images: 520
Processing directory 5
diff_labels 170
labels len (568, 4)
Number of images: 568
Processing directory 6
diff_labels 246
labels len (822, 4)
Number of images: 825
Processing directory 7
diff_labels 189
labels len (631, 4)
Number of images: 631
Processing directory 8
diff_labels 225
labels len (750, 4)
Number of images: 750
Processing directory 9
diff_labels 230
labels len (768, 4)
Number of images: 768
Processing directory 10
diff_labels 207
labels len (691, 4)
Number of images: 691
Processing directory 11
diff_labels 299
labels len (999, 4)
Number of images: 999
Processing directory 12
diff_labels 258
labels len (861, 4)
Number of images: 864
Processing directory 13
d

In [10]:
# trajectory length as a factor of 64

root = "dataset"
file_ending = 'png'
file_name = "data_out.csv"

for i in range(len(os.listdir(root))):
    directory = i + 1
    print("Processing directory", directory)

    csv_file_name = f"{root}/{str(directory)}/{file_name}"
    labels = np.genfromtxt(csv_file_name, delimiter=',', skip_header=1, dtype=np.float32)
    
    d = labels.shape[0]%64
    if d == 0:
        diff_labels = 0
    else:
        diff_labels = 64 - d
 
    # print("labels", labels[-1])
    velocity_values = [0.0, 0.0, 0.0, 0.0]
    
    with open(csv_file_name, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        for i in range(diff_labels):
            csvwriter.writerow(velocity_values)
   

   
    n_images = len([fn for fn in os.listdir(f"./{root}/{str(directory)}") if file_ending in fn])
    print("Number of images:", n_images)
  
    d_img = n_images % 64
    
    if d_img == 0:
        diff = 0
    else:
        diff = 64 - d_img
    print("diff_labels", diff_labels, " diff", diff, " diff_labels%64", (labels.shape[0] + diff_labels)%64, " diff%64", (n_images + diff)%64)
    
    img_file_name = f"{root}/{str(directory)}/Image{str(n_images)}.png"
    img = Image.open(img_file_name)
    for i in range(1, diff + 1):
       
        new_filename = os.path.join(root,str(directory))
        fname =  f"Image{str(i + n_images)}.png"
        file_path = os.path.join(new_filename, fname)
        
        img.save(file_path)
        

Processing directory 1
Number of images: 770
diff_labels 62  diff 62  diff_labels%64 0  diff%64 0
Processing directory 2
Number of images: 704
diff_labels 0  diff 0  diff_labels%64 0  diff%64 0
Processing directory 3
Number of images: 646
diff_labels 58  diff 58  diff_labels%64 0  diff%64 0
Processing directory 4
Number of images: 676
diff_labels 28  diff 28  diff_labels%64 0  diff%64 0
Processing directory 5
Number of images: 738
diff_labels 30  diff 30  diff_labels%64 0  diff%64 0
Processing directory 6
Number of images: 1071
diff_labels 20  diff 17  diff_labels%64 0  diff%64 0
Processing directory 7
Number of images: 820
diff_labels 12  diff 12  diff_labels%64 0  diff%64 0
Processing directory 8
Number of images: 975
diff_labels 49  diff 49  diff_labels%64 0  diff%64 0
Processing directory 9
Number of images: 998
diff_labels 26  diff 26  diff_labels%64 0  diff%64 0
Processing directory 10
Number of images: 898
diff_labels 62  diff 62  diff_labels%64 0  diff%64 0
Processing directory

In [11]:
# Creating sparse subset of the trajectory
root = "dataset"
file_ending = 'png'
file_name = "data_out.csv"
length_root = len(os.listdir(root))

for i in range(len(os.listdir(root))):
    directory = i + 1
    print("Processing directory", directory)
    new_directory = length_root + directory
    print("Creating directory", new_directory)
    new_directory_path = f"{root}/{new_directory}"
    if not os.path.exists(new_directory_path):
        os.makedirs(new_directory_path)
        
   
    csv_file_name = f"{root}/{str(directory)}/{file_name}"
    labels = np.genfromtxt(csv_file_name, delimiter=',', skip_header=1, dtype=np.float32)

    image_files = [fn for fn in os.listdir(f"./{root}/{str(directory)}") if file_ending in fn]
    n_images = len(image_files)
    print("Number of images:", n_images)
    sorted_image_files = list(range(1, n_images + 1))

    # Select every 10th image
    selected_images_files = sorted_image_files[::10]
    selected_images_files = selected_images_files[:44]
    selected_images_files.extend([n_images] * 20)

    selected_labels = labels[::10].tolist()  
    selected_labels = selected_labels[:44]
    
    # print(len(selected_images_files), len(selected_labels))
    
    # last image
    
    velocity_values = [0.0, 0.0, 0.0, 0.0]
    selected_labels.extend([velocity_values] * 20)
    
    # print(selected_images_files, selected_labels)

    for i in range(len(selected_images_files)):
        image_file = selected_images_files[i]
        image_path = f"{root}/{str(directory)}/Image{str(image_file)}.png"
        img = Image.open(image_path)
       
        new_filename =  f"{root}/{new_directory}/Image{str(i + 1)}.png"
       
        img.save(new_filename)
    
    columns = ['vx', 'vy', 'vz', 'omega_z']
    df = pd.DataFrame(selected_labels, columns=columns)
    df.to_csv(f"{root}/{new_directory}/{file_name}", index=False)

Processing directory 1
Creating directory 76
Number of images: 832
Processing directory 2
Creating directory 77
Number of images: 704
Processing directory 3
Creating directory 78
Number of images: 704
Processing directory 4
Creating directory 79
Number of images: 704
Processing directory 5
Creating directory 80
Number of images: 768
Processing directory 6
Creating directory 81
Number of images: 1088
Processing directory 7
Creating directory 82
Number of images: 832
Processing directory 8
Creating directory 83
Number of images: 1024
Processing directory 9
Creating directory 84
Number of images: 1024
Processing directory 10
Creating directory 85
Number of images: 960
Processing directory 11
Creating directory 86
Number of images: 1344
Processing directory 12
Creating directory 87
Number of images: 1152
Processing directory 13
Creating directory 88
Number of images: 1216
Processing directory 14
Creating directory 89
Number of images: 1152
Processing directory 15
Creating directory 90
Numb

In [12]:
# Checking if the dataset has no of velocity and images same

root = "dataset"
file_ending = 'png'
file_name = "data_out.csv"

for i in range(len(os.listdir(root))):
    directory = i + 1
    print("Processing directory", directory)

    csv_file_name = f"{root}/{str(directory)}/{file_name}"
    labels = np.genfromtxt(csv_file_name, delimiter=',', skip_header=1, dtype=np.float32)
    

    image_files = [fn for fn in os.listdir(f"./{root}/{str(directory)}") if file_ending in fn]
    n_images = len(image_files)
    print(f"Number of labels: {len(labels)} Number of images: {n_images}")

Processing directory 1
Number of labels: 832 Number of images: 832
Processing directory 2
Number of labels: 704 Number of images: 704
Processing directory 3
Number of labels: 704 Number of images: 704
Processing directory 4
Number of labels: 704 Number of images: 704
Processing directory 5
Number of labels: 768 Number of images: 768
Processing directory 6
Number of labels: 1088 Number of images: 1088
Processing directory 7
Number of labels: 832 Number of images: 832
Processing directory 8
Number of labels: 1024 Number of images: 1024
Processing directory 9
Number of labels: 1024 Number of images: 1024
Processing directory 10
Number of labels: 960 Number of images: 960
Processing directory 11
Number of labels: 1344 Number of images: 1344
Processing directory 12
Number of labels: 1152 Number of images: 1152
Processing directory 13
Number of labels: 1216 Number of images: 1216
Processing directory 14
Number of labels: 1152 Number of images: 1152
Processing directory 15
Number of labels: 1

In [14]:
root = "dataset"
file_ending = 'png'
file_name = "data_out.csv"
destination = "diff_dataset"

for i in range(len(os.listdir(root))):
    directory = i + 1
    print("Processing directory", directory)
    if not os.path.exists(f"{destination}/{str(directory)}"):
        os.makedirs(f"{destination}/{str(directory)}")
        
    csv_file_name = f"{root}/{str(directory)}/{file_name}"
    labels = np.genfromtxt(csv_file_name, delimiter=',', skip_header=1, dtype=np.float32)
    
    image_files = [fn for fn in os.listdir(f"./{root}/{str(directory)}") if file_ending in fn]
    n_images = len(image_files)

    # Load the goal images
    goal_image = cv2.imread(f'{root}/{str(directory)}/Image{n_images}.png')

    for i in range(1, n_images + 1):
        # Load the current image
        current_image = cv2.imread(f'{root}/{str(directory)}/Image{i}.png')

        # Compute the absolute difference between the images
        difference = cv2.absdiff(current_image, goal_image)

        # Save the result
        cv2.imwrite(f'{destination}/{str(directory)}/Image{i}.png', difference)
    
    columns = ['vx', 'vy', 'vz', 'omega_z']
    df = pd.DataFrame(labels, columns=columns)
    df.to_csv(f"{destination}/{directory}/{file_name}", index=False)




Processing directory 1
Processing directory 2
Processing directory 3
Processing directory 4
Processing directory 5
Processing directory 6
Processing directory 7
Processing directory 8
Processing directory 9
Processing directory 10
Processing directory 11
Processing directory 12
Processing directory 13
Processing directory 14
Processing directory 15
Processing directory 16
Processing directory 17
Processing directory 18
Processing directory 19
Processing directory 20
Processing directory 21
Processing directory 22
Processing directory 23
Processing directory 24
Processing directory 25
Processing directory 26
Processing directory 27
Processing directory 28
Processing directory 29
Processing directory 30
Processing directory 31
Processing directory 32
Processing directory 33
Processing directory 34
Processing directory 35
Processing directory 36
Processing directory 37
Processing directory 38
Processing directory 39
Processing directory 40
Processing directory 41
Processing directory 42
P

In [53]:
root = "dataset"
file_ending = 'png'
file_name = "data_out.csv"
destination = "diff_dataset"

# for i in range(len(os.listdir(root))):
#     directory = i + 1
#     print("Processing directory", directory)
#     if not os.path.exists(f"{destination}/{str(directory)}"):
#         os.makedirs(f"{destination}/{str(directory)}")
        
#     csv_file_name = f"{root}/{str(directory)}/{file_name}"
#     labels = np.genfromtxt(csv_file_name, delimiter=',', skip_header=1, dtype=np.float32)
    
#     image_files = [fn for fn in os.listdir(f"./{root}/{str(directory)}") if file_ending in fn]
#     n_images = len(image_files)

# Load the goal images
goal_image = cv2.imread(f'{root}/1/Image832.png')
print(goal_image.min(), goal_image.max())

current_image = cv2.imread(f'{root}/1/Image1.png')
print(current_image.min(), current_image.max())

# Compute the absolute difference between the images
difference = goal_image - current_image


0 255
0 255


In [51]:
difference.max()

255

In [54]:
cv2.imwrite("diff.png", difference)

True

In [19]:
import pandas as pd
df = pd.DataFrame(difference[0])

In [20]:
df.max().max()

0